<a href="https://colab.research.google.com/github/davidlealo/asistenteabp/blob/main/agenteABP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalar bibliotecas necesarias

In [1]:
! pip install mistralai
!pip install transformers ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.2 MB/s eta 0:00:00


# Solicitar Clave Mistral

In [2]:
from IPython.display import display, HTML, Javascript
from transformers import AutoModelForCausalLM, AutoTokenizer

# Interfaz para ingresar la API key
def request_api_key():
    input_html = """
    <div style="font-family: Arial; margin: 10px; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
        <h3>Proveer API Key</h3>
        <p>Por favor, ingrese su API key de Mistral en el siguiente campo:</p>
        <input type="password" id="apiKeyInput" placeholder="Ingrese su API key aquí" style="width: 100%; padding: 8px; margin-bottom: 10px;">
        <button onclick="storeApiKey()" style="padding: 10px 15px; background-color: #007BFF; color: white; border: none; border-radius: 3px; cursor: pointer;">Guardar API Key</button>
    </div>
    <script>
        function storeApiKey() {
            const apiKey = document.getElementById('apiKeyInput').value;
            if (apiKey) {
                google.colab.kernel.invokeFunction('notebook.store_api_key', [apiKey], {});
                alert('API key almacenada correctamente.');
            } else {
                alert('Por favor, ingrese una API key válida.');
            }
        }
    </script>
    """
    display(HTML(input_html))

# Callback para almacenar la API key en el entorno Python
def store_api_key(api_key):
    global mistral_api_key
    mistral_api_key = api_key
    print("API key almacenada correctamente.")

# Registrar la función en Google Colab
from google.colab import output
output.register_callback('notebook.store_api_key', store_api_key)

# Solicitar la API key al usuario
request_api_key()


API key almacenada correctamente.


# Carga nodelo de Mistral

In [5]:
import requests

# Define la función para interactuar con la API de Mistral
def run_mistral(user_message, model="mistral-large-latest", api_key=None):
    if not api_key:
        raise ValueError("Por favor, proporciona una API key válida para Mistral.")

    # URL base de la API (esto puede variar dependiendo de la documentación de Mistral)
    api_url = f"https://api.mistral.ai/v1/chat/completions"

    # Configura el payload para la solicitud
    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": user_message}
        ]
    }

    # Cabeceras de autenticación
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Envía la solicitud POST
    response = requests.post(api_url, json=payload, headers=headers)

    # Manejo de errores
    if response.status_code != 200:
        raise Exception(f"Error en la API de Mistral: {response.status_code} {response.text}")

    # Devuelve la respuesta generada
    response_data = response.json()
    return response_data["choices"][0]["message"]["content"]



# Prompt master del agente

In [7]:
!wget https://raw.githubusercontent.com/davidlealo/asistenteabp/refs/heads/main/prompt.xml

--2024-12-10 10:45:57--  https://raw.githubusercontent.com/davidlealo/asistenteabp/refs/heads/main/prompt.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5176 (5.1K) [text/plain]
Saving to: ‘prompt.xml’

prompt.xml          100%[===================>]   5.05K  --.-KB/s    in 0s      

2024-12-10 10:45:58 (60.8 MB/s) - ‘prompt.xml’ saved [5176/5176]



# Interactuar con el agente: Opción uno

In [9]:
from IPython.display import display, HTML
from google.colab import output
import requests

# Verifica si la API key ya está almacenada
if 'mistral_api_key' not in globals():
    print("Por favor, asegúrate de haber configurado tu API key antes de continuar.")

# Solicitar datos del proyecto al usuario
def request_project_details():
    html = """
    <div>
        <h3>Detalles del Proyecto</h3>
        <label>Objetivos de Aprendizaje:</label><br>
        <textarea id="learning_objectives" style="width: 100%; height: 60px;"></textarea><br>
        <label>Asignaturas involucradas:</label><br>
        <textarea id="subjects" style="width: 100%; height: 60px;"></textarea><br>
        <label>Otros detalles relevantes:</label><br>
        <textarea id="details" style="width: 100%; height: 60px;"></textarea><br><br>
        <button onclick="submitDetails()">Generar Proyecto</button>
    </div>
    <script>
        function submitDetails() {
            const learningObjectives = document.getElementById("learning_objectives").value;
            const subjects = document.getElementById("subjects").value;
            const details = document.getElementById("details").value;
            google.colab.kernel.invokeFunction("notebook.generate_project", [learningObjectives, subjects, details], {});
        }
    </script>
    """
    display(HTML(html))

# Callback para generar el proyecto
def generate_project(learning_objectives, subjects, details):
    if 'mistral_api_key' not in globals() or not mistral_api_key:
        print("Error: No se encontró una API key válida. Por favor, configura tu API key.")
        return

    # Crear el prompt para Mistral
    prompt = f"""
    Utiliza el siguiente contexto para crear un proyecto basado en ABP:
    Objetivos de Aprendizaje: {learning_objectives}
    Asignaturas: {subjects}
    Detalles Adicionales: {details}

    Sigue el estándar del Gold Standard de PBLWorks y proporciona:
    - Título del Proyecto
    - Pregunta guía
    - Secuencia de Actividades
    - Estrategias de Evaluación
    """

    # Llamar a la API de Mistral
    headers = {
        "Authorization": f"Bearer {mistral_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mistral-large-latest",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post("https://api.mistral.ai/v1/chat/completions", json=data, headers=headers)

    # Procesar respuesta
    if response.status_code == 200:
        project = response.json()["choices"][0]["message"]["content"]
        print("Proyecto generado:")
        print(project)
    else:
        print(f"Error al generar el proyecto: {response.status_code} - {response.text}")

# Registrar la función en Colab
output.register_callback("notebook.generate_project", generate_project)

# Solicitar detalles del proyecto al usuario
request_project_details()


Proyecto generado:
**Título del Proyecto:**
"Filosofía y Ciencia para un Futuro Sostenible"

**Pregunta Guía:**
¿Cómo podemos aplicar principios filosóficos y científicos para abordar el cambio climático y promover la sostenibilidad en nuestra comunidad?

**Secuencia de Actividades:**

1. **Introducción y Contextualización (Semana 1-2):**
   - **Actividad:** Introducción a los conceptos de cambio climático y sostenibilidad.
   - **Objetivo:** Comprender los problemas actuales relacionados con el cambio climático y la sostenibilidad.
   - **Recursos:** Videos documentales, artículos científicos, y discusiones en clase.

2. **Exploración Filosófica (Semana 3-4):**
   - **Actividad:** Análisis de textos filosóficos clásicos y contemporáneos sobre ética y responsabilidad.
   - **Objetivo:** Desarrollar una comprensión crítica y reflexiva de los problemas filosóficos relacionados con la sostenibilidad.
   - **Recursos:** Lecturas filosóficas, debates en clase, y reflexiones escritas.

3. **

# Interactuar con el agente: Opción dos

In [10]:
from IPython.display import display, HTML
from google.colab import output
import requests

# Verificar si la API key ya está configurada
if 'mistral_api_key' not in globals():
    print("Por favor, asegúrate de haber configurado tu API key antes de continuar.")

# Solicitar detalles del proyecto al usuario
def request_project_details():
    html = """
    <div>
        <h3>Detalles del Proyecto</h3>
        <label>Curso:</label><br>
        <input id="grade" type="text" style="width: 100%; padding: 8px;" placeholder="Ejemplo: 5° Básico"/><br><br>
        <label>Objetivos de Aprendizaje:</label><br>
        <textarea id="learning_objectives" style="width: 100%; height: 60px;" placeholder="Escribe los objetivos de aprendizaje aquí..."></textarea><br><br>
        <label>Asignaturas involucradas:</label><br>
        <textarea id="subjects" style="width: 100%; height: 60px;" placeholder="Escribe las asignaturas aquí..."></textarea><br><br>
        <label>Otros detalles relevantes:</label><br>
        <textarea id="details" style="width: 100%; height: 60px;" placeholder="Escribe otros detalles relevantes aquí..."></textarea><br><br>
        <button id="generateButton" style="padding: 10px; background-color: #007BFF; color: white; border: none; border-radius: 5px; cursor: pointer;">Generar Proyecto</button>
    </div>
    <script>
        document.getElementById("generateButton").onclick = function() {{
            const grade = document.getElementById("grade").value;
            const learningObjectives = document.getElementById("learning_objectives").value;
            const subjects = document.getElementById("subjects").value;
            const details = document.getElementById("details").value;
            google.colab.kernel.invokeFunction(
                "notebook.generate_project",
                [grade, learningObjectives, subjects, details],
                {{}}
            );
        }};
    </script>
    """
    display(HTML(html))

# Callback para generar el proyecto
def generate_project(grade, learning_objectives, subjects, details):
    if 'mistral_api_key' not in globals() or not mistral_api_key:
        print("Error: No se encontró una API key válida. Por favor, configura tu API key.")
        return

    # Crear el prompt para Mistral
    prompt = f"""
    Utiliza el siguiente contexto para crear un proyecto basado en ABP:
    Curso: {grade}
    Objetivos de Aprendizaje: {learning_objectives}
    Asignaturas: {subjects}
    Detalles Adicionales: {details}

    Sigue el estándar del Gold Standard de PBLWorks y proporciona:
    - Título del Proyecto
    - Pregunta guía
    - Secuencia de Actividades
    - Estrategias de Evaluación
    """

    # Llamar a la API de Mistral
    headers = {
        "Authorization": f"Bearer {mistral_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mistral-large-latest",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post("https://api.mistral.ai/v1/chat/completions", json=data, headers=headers)

    # Procesar respuesta
    if response.status_code == 200:
        project = response.json()["choices"][0]["message"]["content"]
        print("Proyecto generado:")
        print(project)
    else:
        print(f"Error al generar el proyecto: {response.status_code} - {response.text}")

# Registrar la función en Colab
output.register_callback("notebook.generate_project", generate_project)

# Solicitar detalles del proyecto al usuario
request_project_details()


# Interactuar con el agente: Opción tres

In [11]:
from IPython.display import display, HTML, Markdown
from google.colab import output
import requests

# Verificar si la API key ya está configurada
if 'mistral_api_key' not in globals():
    print("Por favor, asegúrate de haber configurado tu API key antes de continuar.")

# Solicitar detalles del proyecto al usuario
def request_project_details():
    html = """
    <div>
        <h3>Detalles del Proyecto</h3>
        <label>Curso:</label><br>
        <input id="grade" type="text" style="width: 100%; padding: 8px;" placeholder="Ejemplo: 5° Básico"/><br><br>
        <label>Objetivos de Aprendizaje:</label><br>
        <textarea id="learning_objectives" style="width: 100%; height: 60px;" placeholder="Escribe los objetivos de aprendizaje aquí..."></textarea><br><br>
        <label>Asignaturas involucradas:</label><br>
        <textarea id="subjects" style="width: 100%; height: 60px;" placeholder="Escribe las asignaturas aquí..."></textarea><br><br>
        <label>Otros detalles relevantes:</label><br>
        <textarea id="details" style="width: 100%; height: 60px;" placeholder="Escribe otros detalles relevantes aquí..."></textarea><br><br>
        <button id="generateButton" style="padding: 10px; background-color: #007BFF; color: white; border: none; border-radius: 5px; cursor: pointer;">Generar Proyecto</button>
    </div>
    <script>
        document.getElementById("generateButton").onclick = function() {{
            const grade = document.getElementById("grade").value;
            const learningObjectives = document.getElementById("learning_objectives").value;
            const subjects = document.getElementById("subjects").value;
            const details = document.getElementById("details").value;
            google.colab.kernel.invokeFunction(
                "notebook.generate_project",
                [grade, learningObjectives, subjects, details],
                {{}}
            );
        }};
    </script>
    """
    display(HTML(html))

# Callback para generar el proyecto
def generate_project(grade, learning_objectives, subjects, details):
    if 'mistral_api_key' not in globals() or not mistral_api_key:
        print("Error: No se encontró una API key válida. Por favor, configura tu API key.")
        return

    # Crear el prompt para Mistral
    prompt = f"""
    Diseña un proyecto ABP siguiendo el estándar Gold Standard de PBLWorks con esta estructura:
    - **Curso**: {grade}
    - **Objetivos de Aprendizaje**: {learning_objectives}
    - **Asignaturas**: {subjects}
    - **Otros detalles**: {details}

    Estructura del proyecto:
    1. **Título del Proyecto**
    2. **Pregunta guía**
    3. **Fase 1: Construir Conocimiento, Comprensión y Habilidades**
    4. **Fase 2: Desarrollar y Evaluar Productos y Respuestas**
    5. **Fase 3: Presentar Productos y Respuestas**
    """

    # Llamar a la API de Mistral
    headers = {
        "Authorization": f"Bearer {mistral_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mistral-large-latest",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post("https://api.mistral.ai/v1/chat/completions", json=data, headers=headers)

    # Procesar respuesta
    if response.status_code == 200:
        project = response.json()["choices"][0]["message"]["content"]
        display(Markdown(f"# Proyecto Generado\n\n{project}"))
    else:
        print(f"Error al generar el proyecto: {response.status_code} - {response.text}")

# Registrar la función en Colab
output.register_callback("notebook.generate_project", generate_project)

# Solicitar detalles del proyecto al usuario
request_project_details()


In [12]:
from IPython.display import display, HTML, Markdown
from google.colab import output
import requests

# Verifica si la API key ya está almacenada
if 'mistral_api_key' not in globals():
    print("Por favor, asegúrate de haber configurado tu API key antes de continuar.")

# Solicitar datos del proyecto al usuario
def request_project_details():
    html = """
    <div>
        <h3>Detalles del Proyecto</h3>
        <label>Objetivos de Aprendizaje:</label><br>
        <textarea id="learning_objectives" style="width: 100%; height: 60px;"></textarea><br>
        <label>Asignaturas involucradas:</label><br>
        <textarea id="subjects" style="width: 100%; height: 60px;"></textarea><br>
        <label>Otros detalles relevantes:</label><br>
        <textarea id="details" style="width: 100%; height: 60px;"></textarea><br><br>
        <button onclick="submitDetails()">Generar Proyecto</button>
    </div>
    <script>
        function submitDetails() {
            const learningObjectives = document.getElementById("learning_objectives").value;
            const subjects = document.getElementById("subjects").value;
            const details = document.getElementById("details").value;
            google.colab.kernel.invokeFunction("notebook.generate_project", [learningObjectives, subjects, details], {});
        }
    </script>
    """
    display(HTML(html))

# Callback para generar el proyecto
def generate_project(learning_objectives, subjects, details):
    if 'mistral_api_key' not in globals() or not mistral_api_key:
        print("Error: No se encontró una API key válida. Por favor, configura tu API key.")
        return

    # Crear el prompt para Mistral
    prompt = f"""
    Utiliza el siguiente contexto para crear un proyecto basado en ABP:
    Objetivos de Aprendizaje: {learning_objectives}
    Asignaturas: {subjects}
    Detalles Adicionales: {details}

    Sigue el estándar del Gold Standard de PBLWorks y proporciona:
    - Título del Proyecto
    - Pregunta guía
    - Fase 1: Construir Conocimiento, Comprensión y Habilidades
    - Fase 2: Desarrollar y Evaluar Productos y Respuestas
    - Fase 3: Presentar Productos y Respuestas
    """

    # Llamar a la API de Mistral
    headers = {
        "Authorization": f"Bearer {mistral_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mistral-large-latest",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post("https://api.mistral.ai/v1/chat/completions", json=data, headers=headers)

    # Procesar respuesta
    if response.status_code == 200:
        project = response.json()["choices"][0]["message"]["content"]
        # Mostrar el proyecto en formato Markdown
        display(Markdown(f"# Proyecto Generado\n\n{project}"))
    else:
        print(f"Error al generar el proyecto: {response.status_code} - {response.text}")

# Registrar la función en Colab
output.register_callback("notebook.generate_project", generate_project)

# Solicitar detalles del proyecto al usuario
request_project_details()


# Proyecto Generado

### Proyecto Basado en Aprendizaje por Proyectos (ABP)

#### Título del Proyecto
**"Pensamiento Crítico y Sostenibilidad: Una Mirada Filosófica y Científica al Cambio Climático"**

#### Pregunta Guía
**"¿Cómo podemos utilizar el pensamiento crítico y filosófico para abordar y mitigar los efectos del cambio climático en nuestra comunidad y en el mundo?"**

---

### Fase 1: Construir Conocimiento, Comprensión y Habilidades

#### Actividades y Metas:
1. **Introducción al Pensamiento Crítico y Filosófico:**
   - **Actividad:** Lectura y discusión de textos filosóficos clásicos y contemporáneos sobre ética, responsabilidad y sostenibilidad.
   - **Meta:** Desarrollar una comprensión básica de conceptos filosóficos relevantes y su aplicación en problemas actuales.

2. **Comprensión del Cambio Climático:**
   - **Actividad:** Investigación y presentación de informes sobre las causas y efectos del cambio climático.
   - **Meta:** Adquirir conocimientos científicos sobre el cambio climático y sus implicaciones globales.

3. **Evaluación Crítica de Fuentes de Información:**
   - **Actividad:** Análisis de artículos de medios de comunicación sobre cambio climático y sostenibilidad.
   - **Meta:** Aprender a evaluar la validez y confiabilidad de la información científica.

4. **Desarrollo de Habilidades Argumentativas:**
   - **Actividad:** Debates y discusiones en grupo sobre soluciones posibles al cambio climático.
   - **Meta:** Fomentar la autonomía del pensamiento y la capacidad de argumentar de manera efectiva.

---

### Fase 2: Desarrollar y Evaluar Productos y Respuestas

#### Actividades y Metas:
1. **Propuesta de Proyecto de Mitigación:**
   - **Actividad:** Desarrollo de un proyecto de mitigación del cambio climático en la comunidad local.
   - **Meta:** Aplicar conocimientos científicos y filosóficos para proponer soluciones prácticas y sostenibles.

2. **Evaluación de Propuestas:**
   - **Actividad:** Presentación y evaluación de propuestas de proyectos entre pares.
   - **Meta:** Evaluar la viabilidad y el impacto de las propuestas utilizando criterios de validez y confiabilidad.

3. **Creación de Materiales Educativos:**
   - **Actividad:** Creación de materiales educativos (pósteres, videos, folletos) para sensibilizar a la comunidad sobre el cambio climático y las acciones de mitigación.
   - **Meta:** Desarrollar habilidades de comunicación y producción de contenido educativo.

4. **Reflexión y Autocrítica:**
   - **Actividad:** Escritura de un ensayo reflexivo sobre el proceso de aprendizaje y las implicaciones éticas y filosóficas del proyecto.
   - **Meta:** Fomentar la capacidad crítica y reflexiva en la comprensión de problemas filosóficos y científicos.

---

### Fase 3: Presentar Productos y Respuestas

#### Actividades y Metas:
1. **Presentación Final:**
   - **Actividad:** Presentación de los proyectos de mitigación y materiales educativos a la comunidad escolar y local.
   - **Meta:** Compartir los aprendizajes y propuestas con una audiencia más amplia y recibir retroalimentación.

2. **Feria de Sostenibilidad:**
   - **Actividad:** Organización de una feria de sostenibilidad donde los estudiantes presenten sus proyectos y materiales educativos.
   - **Meta:** Promover la conciencia y la acción comunitaria en torno al cambio climático y la sostenibilidad.

3. **Publicación de Resultados:**
   - **Actividad:** Publicación de los ensayos reflexivos y los proyectos de mitigación en una revista escolar o blog.
   - **Meta:** Documentar y difundir los logros y aprendizajes del proyecto.

---

Este proyecto sigue el estándar del Gold Standard de PBLWorks, asegurando que los estudiantes desarrollen habilidades críticas, éticas y reflexivas, mientras abordan problemas filosóficos y científicos relevantes tanto históricamente como en la vida contemporánea.

# Evaluar proyecto

In [ ]:
from IPython.display import display, HTML
from google.colab import output
import requests

# Verificar si la API key ya está configurada
if 'mistral_api_key' not in globals():
    print("Por favor, asegúrate de haber configurado tu API key antes de continuar.")

# Solicitar detalles del proyecto al usuario
def request_project_details():
    html = """
    <div>
        <h3>Detalles del Proyecto</h3>
        <label>Curso:</label><br>
        <input id="grade" type="text" style="width: 100%; padding: 8px;" placeholder="Ejemplo: 5° Básico"/><br><br>
        <label>Objetivos de Aprendizaje:</label><br>
        <textarea id="learning_objectives" style="width: 100%; height: 60px;" placeholder="Escribe los objetivos de aprendizaje aquí..."></textarea><br><br>
        <label>Asignaturas involucradas:</label><br>
        <textarea id="subjects" style="width: 100%; height: 60px;" placeholder="Escribe las asignaturas aquí..."></textarea><br><br>
        <label>Otros detalles relevantes:</label><br>
        <textarea id="details" style="width: 100%; height: 60px;" placeholder="Escribe otros detalles relevantes aquí..."></textarea><br><br>
        <button id="generateButton" style="padding: 10px; background-color: #007BFF; color: white; border: none; border-radius: 5px; cursor: pointer;">Generar Proyecto</button>
    </div>
    <script>
        document.getElementById("generateButton").onclick = function() {{
            const grade = document.getElementById("grade").value;
            const learningObjectives = document.getElementById("learning_objectives").value;
            const subjects = document.getElementById("subjects").value;
            const details = document.getElementById("details").value;
            google.colab.kernel.invokeFunction(
                "notebook.generate_project",
                [grade, learningObjectives, subjects, details],
                {{}}
            );
        }};
    </script>
    """
    display(HTML(html))

# Callback para generar el proyecto
def generate_project(grade, learning_objectives, subjects, details):
    if 'mistral_api_key' not in globals() or not mistral_api_key:
        print("Error: No se encontró una API key válida. Por favor, configura tu API key.")
        return

    # Crear el prompt para Mistral
    prompt = f"""
    Utiliza el siguiente contexto para crear un proyecto basado en ABP:
    Curso: {grade}
    Objetivos de Aprendizaje: {learning_objectives}
    Asignaturas: {subjects}
    Detalles Adicionales: {details}

    Sigue el estándar del Gold Standard de PBLWorks y proporciona:
    - Título del Proyecto
    - Pregunta guía
    - Secuencia de Actividades
    - Estrategias de Evaluación
    """

    # Llamar a la API de Mistral
    headers = {
        "Authorization": f"Bearer {mistral_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "mistral-large-latest",
        "messages": [{"role": "user", "content": prompt}]
    }
    response = requests.post("https://api.mistral.ai/v1/chat/completions", json=data, headers=headers)

    # Procesar respuesta
    if response.status_code == 200:
        project = response.json()["choices"][0]["message"]["content"]
        print("Proyecto generado:")
        print(project)
    else:
        print(f"Error al generar el proyecto: {response.status_code} - {response.text}")

# Registrar la función en Colab
output.register_callback("notebook.generate_project", generate_project)

# Solicitar detalles del proyecto al usuario
request_project_details()


In [8]:
from IPython.display import display, HTML
from google.colab import output

# Ejemplo de proyecto generado por el agente
project_example = """
Título del Proyecto: ¿Cómo respiran los huevos?
Descripción: Este proyecto permite a los estudiantes investigar cómo los pollitos respiran dentro del huevo, utilizando experimentos sobre la porosidad de la cáscara.
Habilidades del siglo XXI: Pensamiento Crítico, Colaboración y Comunicación.
Fases:
1. Planteamiento de la pregunta guía.
2. Investigación experimental utilizando colorantes.
3. Creación de una representación artística.
4. Presentación de resultados y reflexiones.
"""

# Función de Python para recibir la evaluación
def process_evaluation(data):
    print("Evaluación recibida:")
    print(f"Relevancia: {data['relevance']}")
    print(f"Claridad: {data['clarity']}")
    print(f"Comentarios: {data['comments']}")

# Registrar la función en Colab
output.register_callback('notebook.process_evaluation', process_evaluation)

# Generar evaluación interactiva
def display_project_evaluation(project):
    evaluation_html = f"""
    <div style="font-family: Arial; margin: 10px; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
        <h3>Proyecto Generado</h3>
        <p>{project}</p>
        <h4>Evaluación del Proyecto</h4>
        <form id="evaluationForm">
            <label for="relevance">Relevancia del proyecto:</label><br>
            <input type="radio" id="relevance1" name="relevance" value="1"> 1 - Baja<br>
            <input type="radio" id="relevance2" name="relevance" value="2"> 2<br>
            <input type="radio" id="relevance3" name="relevance" value="3"> 3<br>
            <input type="radio" id="relevance4" name="relevance" value="4"> 4<br>
            <input type="radio" id="relevance5" name="relevance" value="5"> 5 - Alta<br><br>

            <label for="clarity">Claridad de las instrucciones:</label><br>
            <input type="radio" id="clarity1" name="clarity" value="1"> 1 - Baja<br>
            <input type="radio" id="clarity2" name="clarity" value="2"> 2<br>
            <input type="radio" id="clarity3" name="clarity" value="3"> 3<br>
            <input type="radio" id="clarity4" name="clarity" value="4"> 4<br>
            <input type="radio" id="clarity5" name="clarity" value="5"> 5 - Alta<br><br>

            <label for="comments">Comentarios adicionales:</label><br>
            <textarea id="comments" name="comments" rows="4" cols="50"></textarea><br><br>

            <button type="button" onclick="submitEvaluation()">Enviar Evaluación</button>
        </form>
    </div>
    <script>
        function submitEvaluation() {{
            const relevance = document.querySelector('input[name="relevance"]:checked')?.value || "Sin seleccionar";
            const clarity = document.querySelector('input[name="clarity"]:checked')?.value || "Sin seleccionar";
            const comments = document.getElementById('comments').value || "Sin comentarios";

            // Enviar datos a Python
            google.colab.kernel.invokeFunction('notebook.process_evaluation',
                [{{relevance, clarity, comments}}],
                {{}});

            alert("Evaluación enviada. Revisa los resultados en el notebook.");
        }}
    </script>
    """
    display(HTML(evaluation_html))

# Mostrar la evaluación del proyecto
display_project_evaluation(project_example)
